In [30]:
import pandas as pd
import os
import matplotlib.pyplot as plt
from scipy.stats import zscore
from scipy.signal import medfilt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from tqdm import tqdm  # Para la barra de progreso
import numpy as np

In [31]:
# Ruta de la carpeta deep_mq
deep_folder = 'data\\lunar\\training\\data\\S12_GradeA\\deep_mq'


In [32]:
def clean_and_detect_anomalies(folder_path, z_threshold=3.0):
    all_data = []  # Lista para almacenar DataFrames
    for filename in os.listdir(folder_path):
        if filename.endswith('.csv'):
            file_path = os.path.join(folder_path, filename)
            data_df = pd.read_csv(file_path)  # Cargar el archivo CSV
            
            # Asegúrate de que las columnas necesarias estén presentes
            if 'velocity(m/s)' in data_df.columns and 'time_abs(%Y-%m-%dT%H:%M:%S.%f)' in data_df.columns:
                # Convertir la columna de tiempo a datetime
                data_df['time_abs(%Y-%m-%dT%H:%M:%S.%f)'] = pd.to_datetime(data_df['time_abs(%Y-%m-%dT%H:%M:%S.%f)'])

                # Aplicar un filtro de mediana para limpiar el ruido
                data_df['filtered_velocity'] = medfilt(data_df['velocity(m/s)'], kernel_size=5)

                # Calcular el Z-score en los datos filtrados
                data_df['z_score'] = zscore(data_df['filtered_velocity'])

                # Detectar anomalías basadas en el Z-score
                data_df['anomaly'] = (data_df['z_score'].abs() > z_threshold)

                # Agregar el DataFrame a la lista
                all_data.append(data_df)
    
    # Combinar todos los DataFrames en uno solo
    combined_data = pd.concat(all_data, ignore_index=True) if all_data else pd.DataFrame()
    
    # Asegúrate de que los datos combinados no estén vacíos
    if combined_data.empty:
        print("No se encontraron datos para combinar.")
    else:
        print("Datos combinados exitosamente.")
    
    return combined_data



In [33]:
combined_data = clean_and_detect_anomalies(deep_folder, z_threshold=3.0)


Datos combinados exitosamente.


In [34]:
print(combined_data.head(100))


   time_abs(%Y-%m-%dT%H:%M:%S.%f)  time_rel(sec)  velocity(m/s)  \
0      1970-04-26 00:00:00.660000       0.000000  -1.583565e-16   
1      1970-04-26 00:00:00.810943       0.150943  -1.872995e-16   
2      1970-04-26 00:00:00.961887       0.301887  -2.007235e-16   
3      1970-04-26 00:00:01.112830       0.452830  -1.989497e-16   
4      1970-04-26 00:00:01.263774       0.603774  -1.857298e-16   
..                            ...            ...            ...   
95     1970-04-26 00:00:14.999623      14.339623   6.331102e-16   
96     1970-04-26 00:00:15.150566      14.490566   1.656111e-15   
97     1970-04-26 00:00:15.301509      14.641509   1.671568e-15   
98     1970-04-26 00:00:15.452453      14.792453   4.126717e-16   
99     1970-04-26 00:00:15.603396      14.943396  -1.436428e-15   

    filtered_velocity   z_score  anomaly  
0       -1.583565e-16  0.002283    False  
1       -1.872995e-16  0.002283    False  
2       -1.872995e-16  0.002283    False  
3       -1.872995e-16  

In [35]:
def train_model(data_df, n_samples=5000):
    if data_df.empty:
        print("El DataFrame está vacío. No se puede entrenar el modelo.")
        return None, None, None  # Retorna tres None si no hay datos
    
    # Filtrar los datos de anomalías y normales
    anomalies = data_df[data_df['anomaly'] == True]
    normal_data = data_df[data_df['anomaly'] == False]

    # Asegurarse de que haya suficientes datos de anomalías
    if len(anomalies) < n_samples // 2:
        print("No hay suficientes datos de anomalías para entrenar.")
        return None, None, None  # Retorna tres None si no hay suficientes datos
    
    # Número de ejemplos que tomaremos de cada clase
    n_anomalies = min(len(anomalies), n_samples // 2)
    n_normals = n_samples - n_anomalies

    # Seleccionar ejemplos aleatorios
    sampled_anomalies = anomalies.sample(n=n_anomalies, random_state=42)
    sampled_normals = normal_data.sample(n=n_normals, random_state=42)

    # Combinar los datos muestreados
    sampled_data = pd.concat([sampled_anomalies, sampled_normals])

    # Extraer características y etiquetas
    features = sampled_data[['filtered_velocity', 'z_score']]  # Características
    labels = sampled_data['anomaly'].astype(int)  # Convertir booleano a entero (0 o 1)

    # Dividir el conjunto de datos en entrenamiento y prueba
    X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

    # Inicializar el modelo
    model = RandomForestClassifier()

    # Entrenar el modelo
    model.fit(X_train, y_train)

    # Realizar predicciones
    y_pred = model.predict(X_test)

    # Evaluar el modelo
    print(classification_report(y_test, y_pred))
    
    return X_test, y_test, y_pred  # Devuelve los datos de prueba y las predicciones




In [36]:
X_test, y_test, y_pred = train_model(combined_data, n_samples=5000)



              precision    recall  f1-score   support

           0       1.00      1.00      1.00       500
           1       1.00      1.00      1.00       500

    accuracy                           1.00      1000
   macro avg       1.00      1.00      1.00      1000
weighted avg       1.00      1.00      1.00      1000



In [37]:
# Asegúrate de que las columnas necesarias estén presentes
if 'velocity(m/s)' in combined_data.columns and 'time_abs(%Y-%m-%dT%H:%M:%S.%f)' in combined_data.columns:
    # Convertir la columna de tiempo a datetime
    try:
        combined_data['time_abs(%Y-%m-%dT%H:%M:%S.%f)'] = pd.to_datetime(combined_data['time_abs(%Y-%m-%dT%H:%M:%S.%f)'], errors='coerce')
        if combined_data['time_abs(%Y-%m-%dT%H:%M:%S.%f)'].isnull().any():
            print(f"Se encontraron fechas no válidas en {filename}. Algunas filas se han convertido a NaT.")
    except Exception as e:
        print(f"Error al convertir fechas en {}: {e}")



SyntaxError: f-string: empty expression not allowed (1948614886.py, line 9)